# Lightweight networks and MobileNet

We have seen that complex networks require significant computational resources, such as GPU, for training, and also for fast inference. However, it turns out that a model with significantly smaller number of parameters in most cases can still be trained to perform reasonably well. In other words, increase in the model complexity typically results in small (non-proportional) increase in the model performance.

We have observed this in the beginning of the module when training MNIST digit classification. The accuracy of simple dense model was not significantly worse than that of a powerful CNN. Increasing the number of CNN layers and/or number of neurons in the classifier allowed us to gain a few percents of accuracy at most.

This leads us to the idea that we can experiment with Lightweight network architectures in order to train faster models. This is especially important if we want to be able to execute our models on mobile devices.

This module will rely on the Cats and Dogs dataset that we have downloaded in the previous unit. First we will make sure that the dataset is available.

我们已经看到，复杂的网络需要大量的计算资源，例如 GPU，用于训练和快速推理。 然而，事实证明，在大多数情况下，参数数量明显减少的模型仍然可以训练得相当好。 换句话说，模型复杂性的增加通常会导致模型性能的小幅（非比例）增加。

在训练 MNIST 数字分类时，我们在模块的开头观察到了这一点。 简单密集模型的准确性并不比强大的 CNN 差很多。 增加分类器中的 CNN 层数和/或神经元数量最多可以让我们获得百分之几的准确度。

这使我们想到可以尝试轻量级网络架构以训练更快的模型。 如果我们希望能够在移动设备上执行我们的模型，这一点尤为重要。

该模块将依赖于我们在上一单元中下载的猫狗数据集。 首先，我们将确保数据集可用。

In [ ]:
!wget https://raw.githubusercontent.com/MicrosoftDocs/pytorchfundamentals/main/computer-vision-pytorch/pytorchcv.py

In [1]:
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
from torchinfo import summary
import os

from pytorchcv import train, display_dataset, train_long, load_cats_dogs_dataset, validate, common_transform

In [2]:
if not os.path.exists('data/kagglecatsanddogs_5340.zip'):
    !wget -P data -q https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip

dataset, train_loader, test_loader = load_cats_dogs_dataset()

/anaconda/envs/py38_default/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:845: UserWarning: Truncated File Read
  warnings.warn(str(msg))


## MobileNet

In the previous unit, we have seen **ResNet** architecture for image classification. More lightweight analog of ResNet is **MobileNet**, which uses so-called *Inverted Residual Blocks*. Let's load pre-trained mobilenet and see how it works:

在上一单元中，我们看到了用于图像分类的 **ResNet** 架构。 ResNet 的更轻量级模拟是 **MobileNet**，它使用所谓的 *Inverted Residual Blocks*。 让我们加载预训练的 mobilenet 并看看它是如何工作的：

In [3]:
model = torch.hub.load('pytorch/vision:v0.13.0', 'mobilenet_v2', weights='MobileNet_V2_Weights.DEFAULT')
model.eval()
print(model)

Downloading: "https://github.com/pytorch/vision/zipball/v0.13.0" to /home/mslearnuser/.cache/torch/hub/v0.13.0.zip
Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /home/mslearnuser/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth


Let's apply the model to our dataset and make sure that it works.

让我们将该模型应用于我们的数据集并确保它有效。

In [4]:
sample_image = dataset[0][0].unsqueeze(0)
res = model(sample_image)
print(res[0].argmax())

tensor(281)


The result (281) is the ImageNet class number, which we have talked about in the previous unit.

> Note that the number of parameters in MobileNet and full-scale ResNet model differ significantly. In some ways, MobileNet is more compact that VGG model family, which is less accurate. However, reduction in the number of parameters naturally leads to some drop in the model accuracy.


结果 (281) 是我们在上一单元中讨论过的 ImageNet 类号。

> 请注意，MobileNet 和全尺寸 ResNet 模型中的参数数量存在显着差异。 在某些方面，MobileNet 比 VGG 模型系列更紧凑，但精度较低。 然而，参数数量的减少自然会导致模型精度有所下降。


## Using MobileNet for transfer learning

Now let's perform the same transfer learning process as in previous unit, but using MobileNet. First of all, let's freeze all parameters of the model:

现在让我们执行与上一单元相同的迁移学习过程，但使用 MobileNet。 首先，让我们冻结模型的所有参数：

In [ ]:
for x in model.parameters():
    x.requires_grad = False

Then, replace the final classifier. We also transfer the model to our default training device (GPU or CPU):

然后，替换最终分类器。 我们还将模型传输到我们的默认训练设备（GPU 或 CPU）：

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.classifier = nn.Linear(1280,2)
model = model.to(device)
summary(model,input_size=(1,3,244,244))

Now let's do the actual training:

现在让我们进行实际训练：


In [8]:
train_long(model,train_loader,test_loader,loss_fn=torch.nn.CrossEntropyLoss(),epochs=1,print_freq=90)

Epoch 0, minibatch 0: train acc = 0.5, train loss = 0.02309325896203518
Epoch 0, minibatch 90: train acc = 0.9443681318681318, train loss = 0.006317565729329874
Epoch 0, minibatch 180: train acc = 0.9488950276243094, train loss = 0.00590015182178982
Epoch 0, minibatch 270: train acc = 0.9492619926199262, train loss = 0.006072205810969167
Epoch 0, minibatch 360: train acc = 0.9500519390581718, train loss = 0.00641324315374908
Epoch 0, minibatch 450: train acc = 0.9494872505543237, train loss = 0.006945275943189397
Epoch 0, minibatch 540: train acc = 0.9521141404805915, train loss = 0.0067323536617257896
Epoch 0 done, validation acc = 0.98245, validation loss = 0.002347727584838867


## Takeaway

Notice that MobileNet results in almost the same accuracy as VGG-16, and just slightly lower than full-scale ResNet. 

The main advantage of small models, such as MobileNet or ResNet-18 is that they can be used on mobile devices. [Here](https://pytorch.org/mobile/android/) is official example of using ResNet-18 on Android device, and [here](https://heartbeat.fritz.ai/pytorch-mobile-image-classification-on-android-5c0cfb774c5b) is similar example using MobileNet. 

请注意，MobileNet 的精度几乎与 VGG-16 相同，只是略低于全尺寸 ResNet。

MobileNet 或 ResNet-18 等小型模型的主要优点是它们可以在移动设备上使用。 [这里](https://pytorch.org/mobile/android/) 是在 Android 设备上使用 ResNet-18 的官方示例，[这里](https://heartbeat.fritz.ai/pytorch-mobile-image-classification-on-android-5c0cfb774c5b) 是使用 MobileNet 的类似示例。